In [2]:
import pandas as pd
import numpy as np 
import string,os
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=FutureWarning)
tf.random.set_seed(2)

In [3]:
df= pd.read_csv('ArticlesApril2017.csv')
df

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,NaN,58fd41ab7c459f24986dbaa7,710,By ANDREW E. KRAMER,article,Reporting on Gays Who ‘Don’t Exist’,"['Chechnya (Russia)', 'Homosexuality and Bisex...",3,Insider,2,2017-04-24 00:07:04,Unknown,"“I see flies, I see mosquitoes,” said a Cheche...",The New York Times,News,https://www.nytimes.com/2017/04/23/insider/rus...
882,NaN,58fd45a17c459f24986dbaaa,1230,By MATT FLEGENHEIMER and THOMAS KAPLAN,article,The Fights That Could Lead to a Government Shu...,"['Trump, Donald J', 'United States Politics an...",3,National,15,2017-04-24 00:23:53,Politics,The Trump administration wants to use the dead...,The New York Times,News,https://www.nytimes.com/2017/04/23/us/politics...
883,NaN,58fd5c2c7c459f24986dbac3,1424,By NOEL MURRAY,article,"‘The Leftovers’ Season 3, Episode 2: Swedish P...","['Television', 'The Leftovers (TV Program)']",3,Culture,0,2017-04-24 02:00:04,Television,"For all its melancholy, “The Leftovers” rarely...",The New York Times,Review,https://www.nytimes.com/2017/04/23/arts/televi...
884,NaN,58fd5c3d7c459f24986dbac4,1052,By BEN BRANTLEY,article,"Thinking Out Loud, But Why?","['Theater', 'The Antipodes (Play)', 'Baker, An...",3,Culture,1,2017-04-24 02:00:25,Unknown,"In this endlessly fascinating work, Annie Bake...",The New York Times,Review,https://www.nytimes.com/2017/04/23/theater/the...


In [4]:
all_headlines=[]
all_headlines.extend(list(df.headline.values))
all_headlines=[h for h in all_headlines if h!= "Unknown"]
len(all_headlines)

831

In [5]:
all_headlines

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan',
 'Variety Puzzle: Acrostic',
 'They Can Hit a Ball 400 Feet. But Play Catch? That’s Tricky.',
 'In Trump Country, Shock at Trump Budget Cuts',
 'Why Is This Hate Different From All Other Hate?',
 'Pick Your Favorite Ethical Offender',
 'My Son’s Growing Black Pride',
 'Jerks and the Start-Ups They Ruin',
 'Trump  Needs  a Brain',
 'Manhood in the Age of Trump',
 'The Value of a Black College',
 'Initial Description',
 'Rough Estimates',
 'El Pasatiempo Nacional',
 'Cooling Off on a Hot Day at Yankee Stadium',
 'Trump’s Staff Mixed Politics and Paydays',
 'A Virtuoso Rebuilding Act Requires Everyone in Tune',

In [6]:
def cleantxt(txt):
    txt="".join(v for v in txt if v not in string.punctuation).lower()
    txt=txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus=[cleantxt(x) for x in all_headlines]
corpus

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan',
 'variety puzzle acrostic',
 'they can hit a ball 400 feet but play catch thats tricky',
 'in trump country shock at trump budget cuts',
 'why is this hate different from all other hate',
 'pick your favorite ethical offender',
 'my sons growing black pride',
 'jerks and the startups they ruin',
 'trump  needs  a brain',
 'manhood in the age of trump',
 'the value of a black college',
 'initial description',
 'rough estimates',
 'el pasatiempo nacional',
 'cooling off on a hot day at yankee stadium',
 'trumps staff mixed politics and paydays',
 'a virtuoso rebuilding act requires everyone in tune',
 'homeland seaso

In [7]:
tokenizer=Tokenizer()
def getseqoftok(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words=len(tokenizer.word_index)+1
    input_sequences=[]
    for line in corpus:
        token_list=tokenizer.texts_to_sequences([line])[0]
        for i in range(1,len(token_list)):
            n_grams_sequence=token_list[:i+1]
            input_sequences.append(n_grams_sequence)
    return input_sequences,total_words
input_sequences,total_words=getseqoftok(corpus)
input_sequences

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80],
 [6, 80, 1],
 [6, 80, 1, 668],
 [6, 80, 1, 668, 10],
 [6, 80, 1, 668, 10, 669],
 [670, 671],
 [670, 671, 129],
 [670, 671, 129, 672],
 [673, 674],
 [673, 674, 368],
 [673, 674, 368, 675],
 [673, 674, 368, 675, 676],
 [105, 171],
 [105, 171, 8],
 [105, 171, 8, 677],
 [1, 678],
 [1, 678, 679],
 [1, 678, 679, 680],
 [17, 681],
 [17, 681, 227],
 [130, 682],
 [130, 682, 22],
 [130, 682, 22, 683],
 [130, 682, 22, 683, 369],
 [130, 682, 22, 683, 369, 33],
 [684, 685],
 [228, 5],
 [228, 5, 229],
 [228, 5, 229, 80],
 [228, 5, 229, 80, 686],
 [228, 5, 229, 80, 686, 10],
 [228, 5, 229, 80, 686, 10, 55],
 [228, 5, 229, 80, 686, 10, 55, 51],
 [131, 172],
 [131, 172, 230],
 [81, 43],
 [81, 43, 687],
 [81, 43, 687, 2],
 [81, 43, 687,

In [8]:
def genpadseq(inpseq):
    max_sequence_len=max([len(x)for x in inpseq])
    inpseq=np.array(pad_sequences(inpseq,maxlen=max_sequence_len,padding="pre"))
    predictors,label=inpseq[:,:-1],inpseq[:,-1]
    label=to_categorical(label,num_classes=total_words)
    return predictors,label,max_sequence_len
predictors,label,max_sequence_len=genpadseq(input_sequences)

In [9]:
input_len=max_sequence_len -1
model = Sequential()
model.add(Embedding(total_words,10,input_length=input_len))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 10)            24220     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2422)              244622    
                                                                 
Total params: 313242 (1.19 MB)
Trainable params: 313242 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.fit(predictors, label, epochs=100, verbose=2, batch_size=16)

Epoch 1/100
301/301 - 5s - loss: 7.3412 - 5s/epoch - 18ms/step
Epoch 2/100
301/301 - 3s - loss: 6.8846 - 3s/epoch - 10ms/step
Epoch 3/100
301/301 - 3s - loss: 6.7489 - 3s/epoch - 10ms/step
Epoch 4/100
301/301 - 3s - loss: 6.6368 - 3s/epoch - 11ms/step
Epoch 5/100
301/301 - 3s - loss: 6.5087 - 3s/epoch - 10ms/step
Epoch 6/100
301/301 - 3s - loss: 6.3653 - 3s/epoch - 10ms/step
Epoch 7/100
301/301 - 3s - loss: 6.2074 - 3s/epoch - 10ms/step
Epoch 8/100
301/301 - 3s - loss: 6.0343 - 3s/epoch - 10ms/step
Epoch 9/100
301/301 - 3s - loss: 5.8793 - 3s/epoch - 10ms/step
Epoch 10/100
301/301 - 3s - loss: 5.7083 - 3s/epoch - 10ms/step
Epoch 11/100
301/301 - 3s - loss: 5.5510 - 3s/epoch - 10ms/step
Epoch 12/100
301/301 - 3s - loss: 5.4005 - 3s/epoch - 10ms/step
Epoch 13/100
301/301 - 3s - loss: 5.2486 - 3s/epoch - 10ms/step
Epoch 14/100
301/301 - 3s - loss: 5.1016 - 3s/epoch - 11ms/step
Epoch 15/100
301/301 - 3s - loss: 4.9672 - 3s/epoch - 10ms/step
Epoch 16/100
301/301 - 3s - loss: 4.8308 - 3s/epo

In [16]:
def gentext(seedtext,nextword,model,max_sequences_len):
    for _ in range(nextword):
        token_list=tokenizer.texts_to_sequences([seedtext])[0]
        token_list=pad_sequences([token_list],maxlen=max_sequences_len-1,padding='pre')
        predictx=model.predict(token_list)
        classx=np.argmax(predictx,axis=1)
        outputword=""
        for word,index in tokenizer.word_index.items():
            if index==classx:
                outputword=word
                break
        seedtext+=" "+outputword
    return seedtext.title()

In [17]:
print (gentext("united states", 5, model, max_sequence_len))
print (gentext("president trump", 4, model, max_sequence_len))
print (gentext("donald trump", 4, model, max_sequence_len))
print (gentext("india and china", 4, model, max_sequence_len))
print (gentext("new york", 4, model, max_sequence_len))
print (gentext("science and technology", 5, model, max_sequence_len))

1/1 [==============================] - 0s 23ms/step
United States Loudest At Inaugural Risk Equal
1/1 [==============================] - 0s 21ms/step
President Trump Subjects A Small Army
1/1 [==============================] - 0s 22ms/step
Donald Trump Plays By His Own
1/1 [==============================] - 0s 21ms/step
India And China Confirmed After Bruising Yearlong
1/1 [==============================] - 0s 20ms/step
New York Today A Holocaust Survivors
1/1 [==============================] - 0s 20ms/step
Science And Technology Selfempowerment Drug Price Gouging Is
